Experimento GA.01 - senha de tamanho variavel
========================================



## Introdução



Delete este texto e escreva uma breve introdução sobre seu experimento.



## Objetivo



Resolver o problema da senha sem fornecer a informação do tamanho da senha para a função que gera a população.

**Dica**: A função objetivo terá que quantificar em sua métrica tanto se o candidato acertou as letras quanto se acertou o tamanho da senha.


## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
from funcoes import populacao_inicial_senha
from funcoes import funcao_objetivo_pop_senha
from funcoes import selecao_torneio_min
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_senha
from funcoes import gene_letra
import random

## Códigos e discussão



In [2]:
def individuo_senha_tv(letras, senha_verdadeira):
    """Cria um candidato para o problema da senha, sem saber o tamanho da senha real.
    Args:
      letras: letras possíveis de serem sorteadas.
      senha_verdadeira: a senha escolhida pelo usuário
      
    Return:
      Lista com n letras, um chute para tentar chegar à senha real
    """

    candidato = []
    valor_senha_verdadeira = 0
    valor_candidato = 1
    
    for s in senha_verdadeira:
        valor_senha_verdadeira = valor_senha_verdadeira + ord(s)

    while valor_candidato < valor_senha_verdadeira:
        candidato.append(gene_letra(letras))
    
    for i in candidato:
        valor_candidato = valor_candidato + ord(candidato)
        
    return candidato

def populacao_inicial_senha_tv(tamanho, letras, senha_verdadeira):
    """Cria população inicial no problema da senha, sem saber o tamanho da senha real
    Args
      tamanho: tamanho da população.
      letras: letras possíveis de serem sorteadas.
      senha_verdadeira: a senha escolhida pelo usuário
   
   Returns:
      Lista com todos os indivíduos da população no problema da senha.
    """
    populacao = []
    for n in range(tamanho):
        populacao.append(individuo_senha_tv(letras, senha_verdadeira))
    return populacao


def funcao_objetivo_senha_tv(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha
    
    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir
    
    Returns:
      A "distância" entre a senha proposta e a senha verdadeira. Essa distância
      é medida letra por letra. Quanto mais distante uma letra for da que
      deveria ser, maior é essa distância.
      O len da senha real é comparado com o len do individuo
    """
    # a maior diferença pode ser de 25 para cada letra, portanto 100 para a senha inteira
    diferenca = 0 # então a diferença pode estar de 0 à 100
    diferenca_len = abs(len(individuo)-len(senha_verdadeira)) 
    # o maior len que o individuo pode ter é 488, se todas letras forem z
    # o menor len do individuo/senha é 388, se todas letras fossem a
    # a maior diferênça é, portanto, 100, variação de 0 á 100.
    
    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))
        
    resultado = diferenca + diferenca_len
    
    return resultado 

def funcao_objetivo_pop_senha_tv(populacao, senha_verdadeira):
    """Computa a funcao objetivo de uma populaçao no problema da senha.
    Args:
      populacao: lista com todos os individuos da população
      senha_verdadeira: a senha que você está tentando descobrir
      
    Returns:
      Lista contendo os valores da métrica de distância entre senhas.
    """
    resultado = []
    resultado_distancia = []

    for individuo in populacao:
        resultado.append(funcao_objetivo_senha_tv(individuo, senha_verdadeira))

    return resultado

In [3]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 2000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resulvido
## limitar a senha para 4 digitos
SENHA = "core" # o que estou tentando descobrir
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz" # todas letras que podem estar na minha senha
# NUM_GENES = len(SENHA)

In [4]:
# funções locais

def cria_populacao_inicial(tamanho, tamanho_senha):
    return populacao_inicial_senha_tv(tamanho, LETRAS_POSSIVEIS, SENHA)

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_senha_tv(populacao, SENHA)

def funcao_selecao(populacao, fitness):
    return selecao_torneio_min(populacao, fitness, NUM_COMBATENTES_NO_TORNEIO)

def funcao_mutacao(individuo):
    return mutacao_senha(individuo, LETRAS_POSSIVEIS)

In [5]:
populacao = cria_populacao_inicial(TAMANHO_POP, LETRAS_POSSIVEIS)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python
                        # assim, o primeiro indivíduo já substituirá, 
                        # pois vai ser melhor que infinito

print("Progresso da melhor senha já vista:")

while melhor_fitness_ja_visto != 0:
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness
        print("".join(melhor_individuo_ja_visto), "- fitness:", melhor_fitness_ja_visto)

print()
print("Melhor palpite da senha encontrado:")
print("".join(melhor_individuo_ja_visto))

KeyboardInterrupt: 

-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [ ]:
caracteres = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "z"]

for caractere in caracteres:
    numero = ord(caractere)
    print(f"O caractere '{caractere}' converte para o número {numero}")

In [4]:
individuo_senha_tv(LETRAS_POSSIVEIS, SENHA)

KeyboardInterrupt: 